## Harmful Brain Activity Classification 

https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/

   The basis of this dataset follows the analysis of electrical activity in the brain via the usage of an electroencephalogram (EEG) on critically ill patients . The EEG uses small electrodes applied to a patient’s scalp to measure the electrical signals fired between neurons, which are measured in voltages and sourced from the spatial placement of the electrode and the corresponding region of the brain. With these measurements, it is possible to detect abnormalities and seizure activity in the brain of a patient via anomalies in the EEG readings. This data problem provided by Harvard Medical School (HMS) seeks to integrate machine learning algorithms in the analysis of such data, to seek out patterns correlative to harmful brain activity with great efficiency. 

Briefly describe the data.

The information within this dataset 

What type of Machine Learning? Supervised Classification (binary or multiclass) or Regression?